In [1]:
import numpy as np
import pandas as pd
import re
import gensim.downloader as model_api
import statsmodels.api as sm
import sklearn.metrics as skm
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

import random
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense

# 1. Sentiment analysis

Using the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/), we want to do a regression model that predict the ratings are on a 1-10 scale. You have an example train and test set in the `dataset` folder.

### 1.1 Regression Model

Use a feedforward neural network and NLP techniques we've seen up to now to train the best model you can on this dataset

### 1.2 RNN model

Train a RNN to do the sentiment analysis regression. The RNN should consist simply of an embedding layer (to make word IDs into word vectors) a recurrent blocks (GRU or LSTM) feeding into an output layer.

In [ ]:
# Uncomment and run this first code after downloading dataset to get all the files

# import tarfile
# tf = tarfile.open('dataset/aclImdb_v1.tar.gz', "r:gz")
# tf.extractall()
# tf.close()

In [2]:
train = []
test = []
train_review = []
test_review = []
train_rating = []
test_rating = []
train_sent = []
test_sent = []

def list_maker(train_test, pos_neg):
    
    if train_test == "train" and pos_neg == 'pos':
        loc = "aclImdb/train/pos/"
    elif train_test == 'train' and pos_neg == 'neg':
        loc = "aclImdb/train/neg/"
    elif train_test == 'test' and pos_neg == 'pos':
        loc = "aclImdb/test/pos/"
    else:
        loc = "aclImdb/test/neg/"
    
    for i in range(0, 12499):
        for j in range(1, 10):
            try:
                f = open(loc+'%s_%s.txt' % (i, j))
                data = f.read()
            except:
                continue
            if train_test == "train":
                train.append(i)
                train_review.append(data)
                train_rating.append(j)
                if pos_neg == 'pos':
                    train_sent.append(1)
                else:
                    train_sent.append(0)
            else:
                test.append(i)
                test_review.append(data)
                test_rating.append(j)
                if pos_neg == 'pos':
                    test_sent.append(1)
                else:
                    test_sent.append(0)

In [3]:
list_maker('train', 'pos')
list_maker('train', 'neg')
list_maker('test', 'pos')
list_maker('test', 'neg')

In [4]:
train_df = pd.DataFrame({'ID':train, 'Review':train_review, 'Rating':train_rating, 'Sentiment':train_sent}).set_index('ID')
test_df = pd.DataFrame({'ID':test, 'Review':test_review, 'Rating':test_rating, 'Sentiment':test_sent}).set_index('ID')

In [5]:
train_df

,Review,Rating,Sentiment
ID,,,
0,Bromwell High is a cartoon comedy. It ran at t...,9,1
1,"If you like adult comedy cartoons, like South ...",7,1
2,Bromwell High is nothing short of brilliant. E...,9,1
4,FUTZ is the only show preserved from the exper...,8,1
7,Although I didn't like Stanley & Iris tremendo...,7,1
...,...,...,...
12494,There was nothing about this movie that I like...,1,0
12495,"OK, I love bad horror. I especially love horro...",1,0
12496,To be brutally honest... I LOVED watching Seve...,1,0


In [6]:
test_df

,Review,Rating,Sentiment
ID,,,
2,My yardstick for measuring a movie's watch-abi...,7,1
3,How many movies are there that you can think o...,7,1
5,I only went to see this movie because I have a...,7,1
6,I was fortunate enough to see this movie on pr...,7,1
7,Our family (and the entire sold out sneak prev...,9,1
...,...,...,...
12494,This movie was obviously made with a very low ...,2,0
12495,CyberTracker is set in Los Angeles sometime in...,3,0
12496,Eric Phillips (Don Wilson) is a secret service...,3,0


In [7]:
words = train_df.Review.str.split()
words = pd.DataFrame(words.tolist())
replaceDict = dict({
'{':" ", '}':" ", ',':"", '.':" ", '!':" ", '\\':" ", '/':" ", '$':" ", '%':" ",
'^':" ", '?':" ", '\'':" ", '"':" ", '(':" ", ')':" ", '*':" ", '+':" ", '-':" ",
'=':" ", ':':" ", ';':" ", ']':" ", '[':" ", '`':" ", '~':" ",
})

rep = dict((re.escape(k), v) for k, v in replaceDict.items())
pattern = re.compile("|".join(rep.keys()))
def replacer(text):
    return rep[re.escape(text.group(0))]

words = train_df.Review.str.replace(pattern, replacer).str.lower().str.split()
words = pd.DataFrame(words.tolist())
words

,0,1,2,3,4,5,6,7,8,9,...,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518
0,bromwell,high,is,a,cartoon,comedy,it,ran,at,the,...,None,None,None,None,None,None,None,None,None,None
1,if,you,like,adult,comedy,cartoons,like,south,park,then,...,None,None,None,None,None,None,None,None,None,None
2,bromwell,high,is,nothing,short,of,brilliant,expertly,scripted,and,...,None,None,None,None,None,None,None,None,None,None
3,futz,is,the,only,show,preserved,from,the,experimental,theatre,...,None,None,None,None,None,None,None,None,None,None
4,although,i,didn,t,like,stanley,&,iris,tremendously,as,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20247,there,was,nothing,about,this,movie,that,i,liked,it,...,None,None,None,None,None,None,None,None,None,None
20248,ok,i,love,bad,horror,i,especially,love,horror,bad,...,None,None,None,None,None,None,None,None,None,None
20249,to,be,brutally,honest,i,loved,watching,severed,that,s,...,None,None,None,None,None,None,None,None,None,None
20250,i,m,sure,that,the,folks,on,the,texas,louisiana,...,None,None,None,None,None,None,None,None,None,None


In [8]:
model = model_api.load("glove-wiki-gigaword-300")

word_vectors = model.wv

def soft_get(w):
    try:
        return word_vectors[w]
    except KeyError:
        return np.zeros(word_vectors.vector_size)

def map_vectors(row):
    try:
        return np.sum(
            row.loc[words.iloc[0].notna()].apply(soft_get)
        )
    except:
        return np.zeros(word_vectors.vector_size)

emb = pd.DataFrame(words.apply(map_vectors, axis=1).tolist())
emb

<ipython-input-8-e5b5b2e3b9e6>:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  word_vectors = model.wv


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-22.084286,16.694746,-2.415143,-14.841063,-7.464311,4.190967,-0.289277,-5.106480,3.760873,-197.139420,...,-2.031199,-34.052475,-3.506387,0.345735,23.491659,-5.969329,10.096868,-22.715826,-12.451815,5.562483
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-16.515263,0.927775,-6.981287,-16.924288,-6.855952,9.484585,-9.155759,11.153566,13.699842,-200.510237,...,-11.141555,-22.964143,1.434592,7.527663,14.549292,6.405801,6.753412,-16.083237,-7.338444,10.124346
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20248,-17.221717,7.736229,-5.489116,-18.015010,-6.146629,7.013818,-7.566385,8.646930,5.584627,-192.853136,...,-2.436579,-22.558603,-0.561796,5.329777,17.732513,4.175543,7.345599,-19.330603,-11.346761,11.988172
20249,-10.199886,7.659323,-4.634923,-10.346042,-10.050541,0.025960,-8.563902,4.170983,4.856806,-142.473457,...,-4.676579,-14.615202,-8.306254,1.860558,7.264510,-0.764165,9.424790,-7.327550,-4.098274,7.761431
20250,-15.619201,20.506126,-8.568425,-17.800774,-9.581174,11.747716,-7.825003,12.141813,6.439943,-181.086682,...,2.732178,-27.940663,-2.633032,0.330947,8.049236,29.449386,4.213533,-28.495936,-12.922284,13.916322


In [9]:
X = emb
X = sm.add_constant(X)
y = train_df.reset_index().Rating

model = sm.OLS(y, X)
res = model.fit(cov_type="HC1")
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     51.82
Date:                Mon, 03 May 2021   Prob (F-statistic):               0.00
Time:                        19:13:33   Log-Likelihood:                -47419.
No. Observations:               20252   AIC:                         9.544e+04
Df Residuals:                   19951   BIC:                         9.782e+04
Df Model:                         300                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2180      0.040    105.644      0.000       4.140       4.296
0              0.0806      0.021      3.814      0.000       0.039       0.122
1             -0.0546      0.023     -2.414      0.016      -0.099      -0.010
2             -0.0069      0.023     -0.298      0.766      -0.052       0.038
3              0.0104      0.021      0.494      0.622      -0.031       0.051
4             -0.0105      0.021     -0.508      0.611      -0.051       0.030
5             -0.0814      0.021     -3.811      0.000      -0.123      -0.040
6             -0.0161      0.019     -0.828      0.408      -0.054       0.022
7             -0.0118      0.021     -0.561      0.575      -0.053       0.029
8             -0.0876      0.021     -4.203      0.000      -0.128      -0.047
9             -0.0016      0.018     -0.090      0.928      -0.036       0.033
10             0.0398      0.023      1.748      0.080      -0.005       0.084
11            -0.0133      0.020     -0.675      0.500      -0.052       0.025
12             0.0312      0.021      1.456      0.145      -0.011       0.073
13             0.0359      0.022      1.614      0.107      -0.008       0.080
14            -0.0478      0.023     -2.052      0.040      -0.093      -0.002
15             0.0121      0.019      0.650      0.516      -0.024       0.048
16            -0.0074      0.021     -0.350      0.726      -0.049       0.034
17            -0.0486      0.020     -2.451      0.014      -0.088      -0.010
18             0.0588      0.025      2.355      0.019       0.010       0.108
19             0.0175      0.021      0.838      0.402      -0.023       0.059
20            -0.0443      0.022     -2.050      0.040      -0.087      -0.002
21             0.1012      0.021      4.823      0.000       0.060       0.142
22            -0.0144      0.025     -0.580      0.562      -0.063       0.034
23             0.0386      0.023      1.709      0.087      -0.006       0.083
24             0.0170      0.020      0.838      0.402      -0.023       0.057
25            -0.0438      0.021     -2.054      0.040      -0.086      -0.002
26            -0.0563      0.023     -2.433      0.015      -0.102      -0.011
27             0.0327      0.020      1.609      0.108      -0.007       0.073
28            -0.0154      0.022     -0.711      0.477      -0.058       0.027
29            -0.0361      0.020     -1.776      0.076      -0.076       0.004
30            -0.0420      0.017     -2.429      0.015      -0.076      -0.008
31            -0.0464      0.023     -1.991      0.046      -0.092      -0.001
32            -0.0159      0.022     -0.725      0.469      -0.059       0.027
33            -0.0011      0.023     -0.050      0.960      -0.046       0.044
34             0.0302      0.027      1.137      0.256      -0.022       0.082
35

In [10]:
pred = res.predict()

print("Mean absolute error =", round(skm.mean_absolute_error(y, pred), 3)) 
print("Mean squared error =", round(skm.mean_squared_error(y, pred), 3)) 
print("Median absolute error =", round(skm.median_absolute_error(y, pred), 3)) 
print("Explain variance score =", round(skm.explained_variance_score(y, pred), 3)) 
print("R2 score =", round(skm.r2_score(y, pred), 3))

Mean absolute error = 2.121
Mean squared error = 6.328
Median absolute error = 2.079
Explain variance score = 0.291
R2 score = 0.291


In [11]:
LogReg = LogisticRegression(random_state=0).fit(X, y)
LogReg.score(X, y)

C:\Users\gaia_\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.35433537428402134

In [12]:
X, y = (train_df.Review.values, train_df.Sentiment.values)

In [13]:
tk = Tokenizer(lower = True)
tk.fit_on_texts(X)

In [14]:
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 300
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
Check_set = test_df.Review.values
Check_seq = tk.texts_to_sequences(Check_set)
Check_pad = pad_sequences(Check_seq, maxlen = 100, padding = 'post')

check_predict = model.predict_classes(Check_pad, verbose = 0)

check_df = pd.DataFrame(list(zip(test_df.Review.values, test_df.Rating.values, check_predict)), columns = ['Review','Rating','Sentiment'])
check_df.Sentiment = [1 if x == [1] else 0 for x in check_df.Sentiment]
check_df

C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


,Review,Rating,Sentiment
0,My yardstick for measuring a movie's watch-abi...,7,1
1,How many movies are there that you can think o...,7,1
2,I only went to see this movie because I have a...,7,0
3,I was fortunate enough to see this movie on pr...,7,1
4,Our family (and the entire sold out sneak prev...,9,1
...,...,...,...
19983,This movie was obviously made with a very low ...,2,1
19984,CyberTracker is set in Los Angeles sometime in...,3,1
19985,Eric Phillips (Don Wilson) is a secret service...,3,0
19986,Plot Synopsis: Los Angeles in the future. Crim...,4,0


# 2. (evil) XOR Problem

Train an LSTM to solve the XOR problem: that is, given a sequence of bits, determine its parity. The LSTM should consume the sequence, one bit at a time, and then output the correct answer at the sequence’s end. Test the two approaches below:

### 2.1 

Generate a dataset of random <=100,000 binary strings of equal length <= 50. Train the LSTM; what is the maximum length you can train up to with precisison?
    

### 2.2

Generate a dataset of random <=200,000 binary strings, where the length of each string is independently and randomly chosen between 1 and 50. Train the LSTM. Does it succeed? What explains the difference?


In [16]:
def rand_key(p):
    key = []
    for i in range(p):
        key.append(str(random.randint(0,1)))
    return(key)

def binary_dataset(n):
    bin_list = []
    for i in range(0, n):
        bin_list.append(rand_key(50))
    return(bin_list)

def rand_binary_dataset(n):
    rand_bin_list = []
    rand = random.randint(1,50)
    for i in range(0, n):
        rand_bin_list.append(rand_key(rand))
    return(rand_bin_list)

def bin_to_num(s):
    int1 = ""
    for ele in s:
        int1 += ele
        return int(int1)
    
def getParity( n ):
    parity = 0
    while n:
        parity = ~parity
        n = n & (n - 1)
    return abs(parity)

In [17]:
bin_list = binary_dataset(100000)

num = []
for i in range(0, len(bin_list)):
    par = []
    par.append(getParity(bin_to_num(bin_list[i])))
    num.append(par)

training_data = np.array(bin_list, "float32")

target_data = np.array(num, "float32")

model = Sequential()
model.add(Dense(16, input_dim=50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

model.fit(training_data, target_data, epochs=10, verbose=1)

model.predict(training_data).round()

Epoch 1/10
3125/3125 [==============================] - 3s 893us/step - loss: 0.0768 - binary_accuracy: 0.9024 2s - l
Epoch 2/10
3125/3125 [==============================] - 2s 771us/step - loss: 9.0424e-05 - binary_accuracy: 1.0000
Epoch 3/10
3125/3125 [==============================] - 3s 853us/step - loss: 9.0016e-06 - binary_accuracy: 1.0000
Epoch 4/10
3125/3125 [==============================] - 3s 826us/step - loss: 1.3406e-06 - binary_accuracy: 1.0000
Epoch 5/10
3125/3125 [==============================] - 3s 840us/step - loss: 2.2821e-07 - binary_accuracy: 1.0000
Epoch 6/10
3125/3125 [==============================] - 3s 829us/step - loss: 4.1315e-08 - binary_accuracy: 1.0000
Epoch 7/10
3125/3125 [==============================] - 3s 813us/step - loss: 8.5714e-09 - binary_accuracy: 1.0000 1s - loss: 1.
Epoch 8/10
3125/3125 [==============================] - 3s 802us/step - loss: 2.3931e-09 - binary_accuracy: 1.0000
Epoch 9/10
3125/3125 [==============================] - 3s 831u

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [18]:
rand_bin_list = rand_binary_dataset(100000)

rand_num = []
for i in range(0, len(rand_bin_list)):
    rand_par = []
    rand_par.append(getParity(bin_to_num(rand_bin_list[i])))
    rand_num.append(rand_par)

training_data = np.array(rand_bin_list, "float32")

target_data = np.array(rand_num, "float32")

model = Sequential()
model.add(Dense(16, input_dim=50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

model.fit(training_data, target_data, epochs=10, verbose=1)

model.predict(training_data).round()

Epoch 1/10


ValueError: in user code:

    C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\gaia_\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

<font color=green>The randomized lengths of binary numbers does not run properly and seems to cause issue in Keras. Model can't seem to run properly if the entries are not consistent. The first part of the question, every single entry was consistently length 50 whereas the second part has lengths ranging from 1 to 50. Ran the input code and left the error message on purpose to show result.</font>